In [1]:
from Bio import Entrez
from Bio import SeqIO

In [2]:
## Retrieve CoV WGS data from NCBI



#get an id list: this makes a big search and gets a list of id 
 #provide your own mail here
Entrez.email = "jiani.chen@uga.edu"
handle = Entrez.esearch(db='nucleotide', term = ["SARS-CoV-2[All Fields]"],rettype="fasta",retmax='500')
record = Entrez.read(handle)
handle.close()
#print (record["IdList"])
ids = record['IdList']

In [3]:
id_str = ",".join(ids)
handle = Entrez.efetch(db="nucleotide", id=id_str,rettype="fasta")
records = SeqIO.parse(handle, "fasta")
#for rec in records:
#   print("downloaded record " + rec.id)
    
    
 ## get sequence 

SeqIO.write(records, "CoV19.fasta", "fasta")

499

In [4]:

handle2 = Entrez.efetch(db="nucleotide", id=id_str,rettype="gb")
records2 = SeqIO.parse(handle2, "gb")
SeqIO.write(records2, "CoV19.gb", "gb")





500

In [23]:
count = SeqIO.convert("CoV19.gb", "genbank", "CoV19gb.fasta", "fasta")
## filter by length to get WGS 
input_seq_iterator = SeqIO.parse("CoV19.gb", "genbank")
short_seq_iterator = (record for record in input_seq_iterator \
                      if len(record.seq) > 20000
                      if "N" not in record.seq
                     )

SeqIO.write(short_seq_iterator, "CoV19_WGS.fasta", "fasta")


248

In [13]:
## get metadata from gb file
#from Bio.SeqRecord import SeqRecord
#for index, record in enumerate(SeqIO.parse("CoV19.gb", "genbank")):
   # print("index %i, ID = %s, length %i, with %i features"
         # % (index, record.id, len(record.seq), len(record.features)))


In [5]:
# Fetch the entries from Entrez
Entrez.email = "jiani.chen@uga.edu" 
handle3 = Entrez.efetch('nuccore', id=ids, retmode='xml')
response = Entrez.read(handle3)
##From https://www.biostars.org/p/221662/
# Parse the entries to get the country
def extract_countries(entry):
    sources = [feature for feature in entry['GBSeq_feature-table']
               if feature['GBFeature_key'] == 'source']

    for source in sources:
        qualifiers = [qual for qual in source['GBFeature_quals']
                      if qual['GBQualifier_name'] == 'country']
        
        for qualifier in qualifiers:
            yield qualifier['GBQualifier_value']

for entry in response:
    accession = entry['GBSeq_primary-accession']
    for country in extract_countries(entry):
        print(accession, country, sep=',')

MT322424,USA: VA
MT322423,USA: VA
MT322422,USA: VA
MT322421,USA: VA
MT322420,USA: VA
MT322419,USA: VA
MT322418,USA: VA
MT322417,USA: VA
MT322416,USA: VA
MT322415,USA: VA
MT322414,USA: VA
MT322413,USA: VA
MT322412,USA: VA
MT322411,USA: VA
MT322410,USA: VA
MT322409,USA: VA
MT322408,USA: VA
MT322407,USA: VA
MT322406,USA: VA
MT322405,USA: VA
MT322404,USA: VA
MT322403,USA: VA
MT322402,USA: VA
MT322401,USA: VA
MT322400,USA: VA
MT322399,USA: VA
MT322398,USA: VA
MT322397,USA: VA
MT322396,USA: VA
MT322395,USA: VA
MT322394,USA: VA
MT077125,Italy
MT039874,China
MT320891,Iran
MT320538,France
MT300186,USA: North Carolina
MT281530,Iran
MT308700,USA: MI
MT308699,USA: MI
MT308698,USA: MI
MT308697,USA: MI
MT308696,USA: MI
MT308695,USA: MI
MT308694,USA: MI
MT308693,USA: MI
MT308692,USA: MI
MT308704,USA: NC
MT308703,USA: NC
MT308702,USA: NC
MT308701,Tunisia: Tunis
MT304491,USA: TX
MT304490,USA: TX
MT304489,USA: TX
MT304488,USA: RI
MT304487,USA: OR
MT304486,USA: NY
MT304485,USA: NH
MT304484,USA: NH
MT3044

In [7]:
import os,sys

sys.stdout = open('output.csv','wt')
for entry in response:
    accession = entry['GBSeq_primary-accession']
    for country in extract_countries(entry):
        print(accession, country, sep='\t')
        
        
       

In [8]:
import pandas as pd

df = pd.read_csv("output.csv",header=None,sep="\t") 
df.columns = ['acession', 'country_state']
df

,acession,country_state
0,MT322424,USA: VA
1,MT322423,USA: VA
2,MT322422,USA: VA
3,MT322421,USA: VA
4,MT322420,USA: VA
...,...,...
486,MT081068,China: HuaShang
487,MT081067,China: HuaShang
488,MT081066,China: HuaShang
489,MT081065,China: HuaShang


In [9]:
df[['country','state']] = df.country_state.str.split(":",expand=True) 
df

,acession,country_state,country,state
0,MT322424,USA: VA,USA,VA
1,MT322423,USA: VA,USA,VA
2,MT322422,USA: VA,USA,VA
3,MT322421,USA: VA,USA,VA
4,MT322420,USA: VA,USA,VA
...,...,...,...,...
486,MT081068,China: HuaShang,China,HuaShang
487,MT081067,China: HuaShang,China,HuaShang
488,MT081066,China: HuaShang,China,HuaShang
489,MT081065,China: HuaShang,China,HuaShang


In [10]:
##combine metadata to seq
records = SeqIO.parse("CoV19_WGS.fasta", "fasta")
SeqIO.write(records, "CoV19.tab", "tab")
df2 = pd.read_csv("CoV19.tab",sep='\t',names=["taxa", "seq"])
df2.head

<bound method NDFrame.head of            taxa                                                seq
0    MT322424.1  ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
1    MT322420.1  CTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCT...
2    MT322417.1  TAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATC...
3    MT322415.1  ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...
4    MT322412.1  TAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATC...
..          ...                                                ...
243  MT135042.1  ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
244  MT135041.1  ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
245  MT126808.1  ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...
246  MT123290.1  GTTCTTTAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTT...
247  MT093571.1  ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...

[248 rows x 2 columns]>

In [11]:
df2[['acession','version']] = df2.taxa.str.split(".",expand=True) 
df2

,taxa,seq,acession,version
0,MT322424.1,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,MT322424,1
1,MT322420.1,CTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCT...,MT322420,1
2,MT322417.1,TAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATC...,MT322417,1
3,MT322415.1,ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...,MT322415,1
4,MT322412.1,TAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATC...,MT322412,1
...,...,...,...,...
243,MT135042.1,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,MT135042,1
244,MT135041.1,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,MT135041,1
245,MT126808.1,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,MT126808,1
246,MT123290.1,GTTCTTTAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTT...,MT123290,1


In [12]:
## add seq to acession
data=pd.merge(df2, df, how='left', left_on='acession', right_on='acession')

In [13]:
data.to_csv('CoV19_country.csv')